<a href="https://colab.research.google.com/github/leonbenenati/leon_Lighthouse_2023/blob/main/Lighthouse_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem

Para criar um modelo para prever se a máquina vai falhar ou não precisamos dividir a base entre treino e teste para evitar o Overfitting

A váriavel de interesse é a failure_type. Ela apresenta 6 tipos de categorias, não falha e 5 tipos de falhas diferentes, não falha apresenta quase 97% das observações, os tipos diferente de não falha menos de 4%.

Como o interesse é identificar as máquinas possíveis de falha, será somado as falhas. Não falha será 0 e falha será 1

Como a váriavel de interesse assume a forma de 0 e 1 para a previsão estamos num problema de classificação 

Na colunas será transformada a váriavel type em variáveis dummy

Para a divisão de X e y, será mantida a proporcionalidade da váriavel alvo, por causa do desbalanceamento

Após as transformações será utilizado o pacote lazypredict que roda vários modelos bases e compara métricas.

A métrica alvo é F1-score pois estamos com uma classe bastante desbalanceadas e essa metrica leva em conta a precisão e recall nela.

Uma segunda métrica utilizado vai ser a AUC, ela leva em conta a taxa de verdadeiros positivos e taxa de falsos positivos.

Será escolhido dois modelos para otimar seus hiperparâmetros, será utilizado  RepeatedStratifiedKFold e RandomizedSearchCV para isso

Um hiperparâmetros importante nesse problema é class_weight, pois podemos aumentar o peso da classe minonitária e melhorar o desempenho do modelo com isso

Todo esse processo será feito na base de treino e foi descrito durante o código

Após achar o melhor modelo com a base de treino, ele será utilizado para fazer a previsão na base de teste

# Pacotes inciais e ajustes

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/desafio_manutencao_preditiva_treino.csv")
df.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure
3,6,M14865,M,298.1,308.6,1425,41.9,11,No Failure
4,7,L47186,L,298.1,308.6,1558,42.4,14,No Failure


In [ ]:
df['failure_type'].unique()

array(['No Failure', 'Power Failure', 'Tool Wear Failure',
       'Overstrain Failure', 'Random Failures',
       'Heat Dissipation Failure'], dtype=object)

In [ ]:
df['failure_type'].value_counts()/df['failure_type'].shape[0]

No Failure                  0.965202
Heat Dissipation Failure    0.011249
Power Failure               0.009450
Overstrain Failure          0.007800
Tool Wear Failure           0.004500
Random Failures             0.001800
Name: failure_type, dtype: float64

In [ ]:
def auxiliar(x):
    if x == "No Failure":
        return 0
    else:
        return 1

df['failure_type'] = df['failure_type'].apply(auxiliar)
df.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0
2,5,L47184,L,298.2,308.7,1408,40.0,9,0
3,6,M14865,M,298.1,308.6,1425,41.9,11,0
4,7,L47186,L,298.1,308.6,1558,42.4,14,0


In [ ]:
df['failure_type'].value_counts()/df['failure_type'].shape[0]

0    0.965202
1    0.034798
Name: failure_type, dtype: float64

*   A primeira váriavel ajustada para rodar um modelo é a várivel alvo que neste caso é a failure_type. 
*   É uma váriavel categorica que precisa ser transformada em númerica.
*   Está váriavel tem No Failure como uma das categorias e outras 5 formas diferentes de falha, não falha representa mais de 96% da base e as outras somadas menos de 4%, como os vários tipos de falhas tem pouco representaividade na base será somada os tipos de falhas.
*   Depois da transformação 0 representa não falha e 1 falha
*   Como visto na EDA a classe está desbalanceada 




In [ ]:
#Transformação da variável type em dummy
variaveis_cat = ['type']
df = pd.get_dummies(df,columns=variaveis_cat, drop_first=True)
df.head()

,udi,product_id,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type,type_L,type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,1,0
2,5,L47184,298.2,308.7,1408,40.0,9,0,1,0
3,6,M14865,298.1,308.6,1425,41.9,11,0,0,1
4,7,L47186,298.1,308.6,1558,42.4,14,0,1,0


*   A variável type será transformada em dummy pois ela tem 3 categorias diferentes
*   Uma delas foi dropada pois quando as duas são 0 é 1 na outra, reduzindo assim uma coluna desnecessária.


# Divisão de X e y

In [ ]:
#Divisão de X e y
X = df.drop(['failure_type','udi','product_id'], axis = 1)
y = df['failure_type']

*   X são as váriaveis para explicar o y. udi e product_id não serão utilizados pois são identificadores unicos
*   y é a nossa váriavel alvo, se houve falha ou não na máquina
*   stratify=y para manter a proporção de y

In [ ]:
#Divisão de treino e teste 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=30,stratify=y)

In [ ]:
df['failure_type'].value_counts()/df['failure_type'].shape[0]

0   0.97
1   0.03
Name: failure_type, dtype: float64

In [ ]:
y.value_counts()/y.shape[0]

0    0.965202
1    0.034798
Name: failure_type, dtype: float64

*   Na divisão de treino e teste a proporcionalidade de y foi mantida, importante pois se trata de um problema desbalanceado 

# Testes nos modelos bases

Lazypredict é um pacote que roda vários modelos base e comparar as métricas

Com base nessas métricas será escolhido dois modelos para tunar os hiperparametros

In [ ]:
pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [00:05<00:00,  5.40it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
DecisionTreeClassifier,0.98,0.81,0.81,0.98,0.04
BaggingClassifier,0.98,0.78,0.78,0.98,0.13
NearestCentroid,0.72,0.75,0.75,0.81,0.02
LGBMClassifier,0.98,0.75,0.75,0.98,0.17
XGBClassifier,0.98,0.73,0.73,0.98,0.21
RandomForestClassifier,0.98,0.69,0.69,0.97,0.53
LinearDiscriminantAnalysis,0.97,0.68,0.68,0.97,0.05
AdaBoostClassifier,0.98,0.68,0.68,0.97,0.25
LabelSpreading,0.97,0.67,0.67,0.96,1.41


*   DecisionTreeClassifier e LGBMClassifier foram escolhidos para a tunagem com base em F1-score e AUC
*   BaggingClassifier não tem class_weight entre os seus hiperparâmetros 
*   NearestCentroid teve uma acurácia muito baixa, é esperado que os modelos tenham alta acurácia pois as classes estão muito desbalanceadas 

# Tunagem de hiperparâmetros

In [ ]:
# Configuração da Validação
from sklearn.model_selection import RepeatedStratifiedKFold
kf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth': [3,5,7,9,11,15,20],
    'n_estimators':[50,100,150,200,300,400],
    'learning_rate': [0.05,0.1,0.2],
    'reg_alpha': [0,0.1,0.5,0.1,0.2],
    'reg_lambda': [0,0.1,0.5],
    'class_weight':[{0:1,1:70},{0:1,1:71},{0:1,1:72},{0:1,1:73},{0:1,1:74},{0:1,1:75},{0:1,1:76},{0:1,1:77},{0:1,1:78},{0:1,1:79},{0:1,1:80}]
}
modelo=LGBMClassifier()
lgbm=RandomizedSearchCV(modelo, param_distributions = params,
                                   scoring = 'f1', 
                                   cv = kf, 
                                   verbose = 1,
                                   n_iter = 10,
                                   n_jobs = -1)
lgbm.fit(X_train,y_train)

Fitting 100 folds for each of 10 candidates, totalling 1000 fits


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=None),
                   estimator=LGBMClassifier(), n_jobs=-1,
                   param_distributions={'class_weight': [{0: 1, 1: 70},
                                                         {0: 1, 1: 71},
                                                         {0: 1, 1: 72},
                                                         {0: 1, 1: 73},
                                                         {0: 1, 1: 74},
                                                         {0: 1, 1: 75},
                                                         {0: 1, 1: 76},
                                                         {0: 1, 1: 77},
                                                         {0: 1, 1: 78},
                                                         {0: 1, 1: 79},
                                                         {0: 1, 1: 80}],
                                        'learning_rate':

*   De primeira foi testado class_weight com peso até 100 na classe minonitária com intervalo de 10.
*   O melhor deu 80, então foi testado em seguida 70 a 80 na classe minonitária

In [ ]:
lgbm.best_params_

{'reg_lambda': 0.1,
 'reg_alpha': 0,
 'n_estimators': 300,
 'max_depth': 20,
 'learning_rate': 0.05,
 'class_weight': {0: 1, 1: 77}}

In [ ]:
from sklearn.metrics import f1_score

pred=lgbm.predict(X_test)
f1_lgbm=f1_score(y_test, pred)

print(f' O F1-score do LGBM é {f1_lgbm} ')

 O F1-score do LGBM é 0.6277372262773724 


In [ ]:
from sklearn.metrics import roc_auc_score

preds = lgbm.predict_proba(X_test)[:,1]
roc_lgbm=roc_auc_score(y_test,preds)
print(f' O AUC do LGBM é {roc_lgbm} ')

 O AUC do LGBM é 0.9559147739883109 


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV


params = {'max_depth' : [1,5,15,50,100], 
              'min_samples_split': [1,5,15,50,75,100,125],
              'min_samples_leaf': [1,5,15,50,100],
                 'class_weight':[{0:1,1:1},{0:1,1:10},{0:1,1:20},{0:1,1:30},{0:1,1:40},{0:1,1:50},{0:1,1:60},{0:1,1:70},{0:1,1:80},{0:1,1:90},{0:1,1:100}]
}
modelo=DecisionTreeClassifier()
tree=RandomizedSearchCV(modelo, param_distributions = params,
                                   scoring = 'f1', 
                                   cv = kf, 
                                   verbose = 1,
                                   n_iter = 30,
                                   n_jobs = -1)
tree.fit(X_train,y_train)

Fitting 100 folds for each of 30 candidates, totalling 3000 fits


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=None),
                   estimator=DecisionTreeClassifier(), n_iter=30, n_jobs=-1,
                   param_distributions={'class_weight': [{0: 1, 1: 1},
                                                         {0: 1, 1: 10},
                                                         {0: 1, 1: 20},
                                                         {0: 1, 1: 30},
                                                         {0: 1, 1: 40},
                                                         {0: 1, 1: 50},
                                                         {0: 1, 1: 60},
                                                         {0: 1, 1: 70},
                                                         {0: 1, 1: 80},
                                                         {0: 1, 1: 90},
                                                         {0: 1, 1: 100}],
                                     

In [ ]:
tree.best_params_

{'min_samples_split': 5,
 'min_samples_leaf': 15,
 'max_depth': 50,
 'class_weight': {0: 1, 1: 1}}

In [ ]:
from sklearn.metrics import f1_score

pred=tree.predict(X_test)
f1_tree=f1_score(y_test, pred)

print(f' O F1-score da Árvore de decisão é {f1_tree} ')

 O F1-score da Árvore de decisão é 0.6166666666666666 


In [ ]:
from sklearn.metrics import roc_auc_score

preds = tree.predict_proba(X_test)[:,1]
auc_tree=roc_auc_score(y_test,preds)

print(f' O AUC da Árvore de decisão é {auc_tree} ')

 O AUC da Árvore de decisão é 0.9331989346748539 


*   Após tunar os hiperparâmetro da Árvore de decisão e LGBM foi verificado que tanto o F1-score e AUC foram melhores no LGBM

## Fazendo predição na base de teste

O modelo escolhido foi o LGBM

será feito as transformações necessárias na base de teste para fazer a previsão

In [ ]:
teste = pd.read_csv("/content/desafio_manutencao_preditiva_teste.csv")
teste.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
0,446,L47625,L,297.50,308.60,1793,26.70,70
1,7076,L54255,L,300.70,310.50,1536,47.40,192
2,1191,L48370,L,297.20,308.40,1460,42.10,41
3,2618,L49797,L,299.40,309.10,1670,35.90,68
4,5067,L52246,L,304.10,313.10,1550,30.90,9


In [ ]:
#Transformação da variável type em dummy
variaveis_cat = ['type']
teste = pd.get_dummies(teste,columns=variaveis_cat, drop_first=True)
teste.head()

,udi,product_id,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,type_L,type_M
0,446,L47625,297.50,308.60,1793,26.70,70,1,0
1,7076,L54255,300.70,310.50,1536,47.40,192,1,0
2,1191,L48370,297.20,308.40,1460,42.10,41,1,0
3,2618,L49797,299.40,309.10,1670,35.90,68,1,0
4,5067,L52246,304.10,313.10,1550,30.90,9,1,0


In [ ]:
x_final = teste.drop(['udi','product_id'], axis = 1)
x_final.head()

,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,type_L,type_M
0,297.50,308.60,1793,26.70,70,1,0
1,300.70,310.50,1536,47.40,192,1,0
2,297.20,308.40,1460,42.10,41,1,0
3,299.40,309.10,1670,35.90,68,1,0
4,304.10,313.10,1550,30.90,9,1,0


In [ ]:
pred_final=lgbm.predict(x_final)
pred_final

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
subm = pd.DataFrame()
subm['predictedValues'] = pred_final
subm

,predictedValues
0,0
1,0
2,0
3,0
4,0
...,...
3328,0
3329,0
3330,0
3331,0


In [ ]:
subm['predictedValues']= subm['predictedValues'].map({0:'No Failure', 1:'Failure'})
subm.head()

,predictedValues
0,No Failure
1,No Failure
2,No Failure
3,No Failure
4,No Failure


In [ ]:
subm.to_csv('predicted.csv', index = False)